In [1]:
from gis_preprocess import pt_gis_train_test_split
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

In [2]:
# train, val, test = pt_gis_train_test_split(image_type="hsv_with_ir")

In [3]:
train, val, test = pt_gis_train_test_split(image_type="full_channel")

Processing file /scratch/mzvyagin/Ephemeral_Channels/Imagery/vhr_2014_refl.img....


In [ ]:
train[0][0].shape

In [4]:
with_streams = []
for dataset in [train, val, test]:
    for i in tqdm(dataset):
        if 1 in i[1]:
            with_streams.append(i)

100%|████████████████████████████████████████████████████████████████████████| 26433/26433 [00:02<00:00, 10674.62it/s]


In [5]:
len(with_streams)/(len(train)+len(val)+len(test))

0.05412215126660815

In [ ]:
from rasterio.plot import show

In [ ]:
show(with_streams[0][0][3].float())

In [ ]:
plt.imshow(with_streams[0][0][1:4, :, :].swapaxes(0, 2).float())
# plt.imshow(with_streams[0][0][3].float())

In [ ]:
plt.imshow(torch.rot90(with_streams[0][1].float()))

In [ ]:
saved_sample = None
for i in test:
    if 1 in i[1]:
        saved_sample = i
        break

In [ ]:
saved_sample

In [ ]:
plt.imshow(saved_sample[1].float())

In [ ]:
len(train) + len(val) + len(test)

In [ ]:
len(train)

In [ ]:
len(val)

In [ ]:
len(test)

In [ ]:
model = torch.load('/tmp/mzvyagin/ephemeral_streams_model.pkl')
out = model.eval()

In [ ]:
out = model(saved_sample[0].float().unsqueeze(0).cuda())
out = torch.nn.Sigmoid()(out)
out = np.rint(out.detach().cpu().squeeze().numpy())

In [ ]:
plt.imshow(out)

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
# generate 512
windows = []
for i in tqdm(list(chunks(train, 4))):
    top = np.hstack([i[0], i[1]])
    bottom = np.hstack([i[1], i[2]])
    full = np.vstack([top, bottom])
    windows.append(full)

In [ ]:
windows[0]